In [1]:
import requests
from bs4 import BeautifulSoup
from akshare.utils import demjson
from akshare.utils.tqdm import get_tqdm
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
start_year = "2020"

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/89.0.4389.90 Safari/537.36",
    }
headers_bk = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/89.0.4389.90 Safari/537.36",
            "Referer": "https://q.10jqka.com.cn",
            "Host": "d.10jqka.com.cn",
        }

link_code_map = []
bk_data_ls = []
today_data_ls = []

for suffix in ["gn/","thshy/"]:
    url = f"https://q.10jqka.com.cn/{suffix}"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, features="lxml")
    divs = soup.find_all(name="div", attrs={"class": "cate_items"})
    for div in divs:
        # 在每个div中查找所有的a标签
        links = div.find_all('a') 
        for link in links:
            # 获取href属性
            href = link.get('href') 
            # 获取文本内容
            text = link.text 
            link_code_map.append((href,  text))


for symbol_url, name in link_code_map[:2]:
    r = requests.get(symbol_url, headers=headers)
    soup = BeautifulSoup(r.text, features="lxml")
    symbol_code = soup.find(name="div", attrs={"class": "board-hq"}).find("span").text
    big_df = pd.DataFrame()
    current_year = datetime.now().year
    dd_texts = [dd.text for dd in soup.find_all("dd")] 
    close = soup.find("span",  class_="board-xj arr-fall").text 
    open, low, high, volume, amount = dd_texts[0], dd_texts[2], dd_texts[3], dd_texts[4], dd_texts[9]
    today_data_ls.append((name, symbol_code, open, high, low, close, volume, amount))

    tqdm = get_tqdm()
    for year in tqdm(range(int(start_year), current_year + 1), leave=False):
        url = f"https://d.10jqka.com.cn/v4/line/bk_{symbol_code}/01/{year}.js"
        r = requests.get(url, headers=headers_bk)
        data_text = r.text
        try:
            demjson.decode(data_text[data_text.find("{") : -1])
        except:  # noqa: E722
            continue
        temp_df = demjson.decode(data_text[data_text.find("{") : -1])
        temp_df = pd.DataFrame(temp_df["data"].split(";"))
        temp_df = temp_df.iloc[:, 0].str.split(",", expand=True)
        big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)
    if big_df.columns.shape[0] == 12:
        big_df.columns = ["date","open","high","low","close","volume","amount","_","_","_","_","_",]
    else:
        big_df.columns = ["date","open","high","low","close","volume","amount","_","_","_","_",]
    big_df = big_df[["date","open","high","low","close","volume","amount",]]
    big_df["date"] = pd.to_datetime(big_df["date"], errors="coerce").dt.date
    big_df["open"] = pd.to_numeric(big_df["open"], errors="coerce")
    big_df["high"] = pd.to_numeric(big_df["high"], errors="coerce")
    big_df["low"] = pd.to_numeric(big_df["low"], errors="coerce")
    big_df["close"] = pd.to_numeric(big_df["close"], errors="coerce")
    big_df["volume"] = pd.to_numeric(big_df["volume"], errors="coerce")
    big_df["amount"] = pd.to_numeric(big_df["amount"], errors="coerce")
    bk_data_ls.append((name, big_df))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
print(bk_data_ls[0][0])
print(bk_data_ls[0][1].tail())

阿尔茨海默概念
           date      open      high       low     close     volume  \
472  2025-09-02  1353.379  1353.603  1320.547  1330.656  982424350   
473  2025-09-03  1333.546  1343.817  1317.655  1322.170  813860100   
474  2025-09-04  1326.883  1337.293  1294.136  1313.114  837025920   
475  2025-09-05  1312.889  1335.951  1290.293  1335.951  693464640   
476  2025-09-08  1336.437  1356.452  1335.287  1353.549  705241630   

           amount  
472  1.789247e+10  
473  1.646862e+10  
474  1.476401e+10  
475  1.385096e+10  
476  1.465946e+10  
